<!DOCTYPE html>
<html>

<body>
    <h2><span style="color: lightblue;">Influx v3</span></h2>
    <h3><strong>Desc:</strong></h3>
    <ul>
        <li>Write metrics to Influx Database.</li>
        <li>Read Metrics from a bucket.</li>
    </ul>
    <h3>Comments:</h3>
    <ul>
        <li>This notebook is still in <strong>DRAFT</strong> mode.</li>
        <li>This notebook was developed for Influx cloud v3. Do NOT use it with v2</li>
    </ul>
</body>
</html>

## Pre Req
1. Open an Iflux DB cloud account. Can be the free one. https://www.influxdata.com/get-influxdb/    
2. Create a new Bucket. For ex. ```Tables_bucket```.   
3. To learn about the InfluxDB terminology, rad here: https://docs.influxdata.com/influxdb/v2/get-started/ 
   
The rest of the flow follows the built-in samples
1. Open the "Load Data" option of "Client" 
2. Select "Python"
3. Follow the instructions. Step "install dependencies" is pip install influxdb3-python, pip install pandas

## Get Token. 
Without a token, the client code can't call the API. To get a Token, log in to the cloud console, Load Data, API Tokens, Generate API Token. 

## Initialize Client
Copy the code. I named the organization in Influx "Dev"

Notice. The original code, proivded by Influx returns an error: ```[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)``` . To solve it, add the coomand ``` ssl_ca_cert=certifi.where()```. Resource: # https://stackoverflow.com/questions/69401104/influxdb-2-0-certificate-verify-failed-certificate-has-expired-ssl-c1129
   

In [2]:
import os, time
from influxdb_client_3 import InfluxDBClient3, Point
import certifi # we need it to support their certification problems. 
import pandas as pd
from datetime import timedelta

# token = "ylThfbJSW4MDfbcEIyNQsS2vnHOBjz-Uanc0KgHAZIcpBSRPc1WaPRwVKmdNzPj8H-HNQyEaN32dqrC10zjVWg=="
token = "PjObG70ZggAu78U5hN9awHq5pOk6GOtAsu7Fp4JROzdHmllRCMrlQh0r9owylgKR0A_Ki7-EYgzz03TNad0tqw=="
org = "Metis-v3"
host = "https://eu-central-1-1.aws.cloud2.influxdata.com"
bucket = "test"

try: 
    client = InfluxDBClient3(host=host, token=token, org=org, ssl_ca_cert=certifi.where())
except ExceptionType as e:
    print (str(e))

## Execute a Simple Query - SQL 
v3 supports SQL. v2 couldn't.

In [3]:
query = """SELECT *
FROM 'census'
WHERE time >= now() - interval '168 hours'
AND ('bees' IS NOT NULL OR 'ants' IS NOT NULL)"""

# Execute the query
table = client.query(query=query, database="Tables_Bucket", language='sql') 

# Convert to dataframe
df = table.to_pandas().sort_values(by="time")
column_names = df.columns
print(column_names)
print(df)




ArrowInvalid: Flight returned invalid argument error, with message: bucket "Tables_Bucket" not found. gRPC client debug context: UNKNOWN:Error received from peer ipv4:3.123.149.45:443 {created_time:"2023-10-22T15:03:14.792118+03:00", grpc_status:3, grpc_message:"bucket \"Tables_Bucket\" not found"}. Client context: IOError: Server never sent a data message. Detail: Internal

## Query the data - Group by time ranges
You can always use a good olf Flux to run a GROUP BY query. Group the data every 5 min. 
It the query doesn't return any data, that means that no data was inserted to this Influx Bucket. See one of the cells above how to insert data.   

- Example 1 - running using the Influx CLI ( ```brew install influxdb-cli```). Apparently it still works with Flux. 
- Exampel 2 - Running using Python. It can't use Flux anymore. Only InfluQL ( https://docs.influxdata.com/influxdb/v1/query_language/, supported languages now are only SQL or InfluxQL: https://docs.influxdata.com/influxdb/cloud-dedicated/reference/client-libraries/v3/python/#functions)

## Testing on V3 - real world data
Step 1: A single Query ID, last 14 days, group hourly. 

In [10]:
# InfluxQL Query (NOT Flux)
# The last 14 days of a specific query ID, group by hour. 

### THIS QUERY DOESN'T WORK - the query should show the hourly diff of every hour. It shows 0.
### I have no idea what fill(NULL) dows but no impact. 
query = """
SELECT time,
    apiKey,
    db,
    query_id, 
    max(calls)
   -- DERIVATIVE(max(calls))
FROM QUERY_DETAILS
WHERE time >= '2023-10-5T00:00:00Z'
AND apiKey = 'mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm'
-- AND host = 'database-2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com'
-- AND db = 'platform-v2'

GROUP BY time(60m)  fill(null)
"""
table = client.query(query=query, database=bucket, mode ="all", language="influxql")
dataframe = table.to_pandas() # This one automatically eliminitaes the NULL values. Not good. 
print (dataframe)





    iox::measurement                time  \
0      QUERY_DETAILS 2023-10-05 00:00:00   
1      QUERY_DETAILS 2023-10-05 01:00:00   
2      QUERY_DETAILS 2023-10-05 02:00:00   
3      QUERY_DETAILS 2023-10-05 03:00:00   
4      QUERY_DETAILS 2023-10-05 04:00:00   
..               ...                 ...   
416    QUERY_DETAILS 2023-10-22 08:00:00   
417    QUERY_DETAILS 2023-10-22 09:00:00   
418    QUERY_DETAILS 2023-10-22 10:00:00   
419    QUERY_DETAILS 2023-10-22 11:00:00   
420    QUERY_DETAILS 2023-10-22 12:00:00   

                                       apiKey           db  \
0    mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm  platform-v2   
1    mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm  platform-v2   
2    mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm  platform-v2   
3    mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm  platform-v2   
4    mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm  platform-v2   
..                                        ...          ...   
416  mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt

### InfluxQL queries for the widgets of the Database Activity

In [11]:
# The last 14 days of a specific query ID, group by hour. 

query_first_100_rows = """
SELECT time,
    apiKey,
    db,
    host,
    value
FROM ROWS_FETCHED
WHERE time >= '2023-10-15T00:00:00Z'
AND apiKey = 'mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm'
AND host = 'database-2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com'
AND db = 'platform-v2'
LIMIT 100
"""

query_rows_read = """
SELECT time,
    apiKey,
    host,
    db,
    max(value)
FROM QUERY_DETAILS
WHERE time >= '2023-10-15T00:00:00Z'
AND apiKey = 'mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm'
--AND host = 'database-2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com'
-- AND db = 'platform-v2'
GROUP BY time(60m)  fill(null)
"""
# table = client.query(query=query_first_100_rows, database=bucket, mode ="all", language="influxql")
table = client.query(query=query_rows_read, database=bucket, mode ="all", language="influxql")
dataframe = table.to_pandas() # This one automatically eliminitaes the NULL values. Not good. 
dataframe



,iox::measurement,time,apiKey,host,db,max
0,QUERY_DETAILS,2023-10-15 00:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
1,QUERY_DETAILS,2023-10-15 01:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
2,QUERY_DETAILS,2023-10-15 02:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
3,QUERY_DETAILS,2023-10-15 03:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
4,QUERY_DETAILS,2023-10-15 04:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
...,...,...,...,...,...,...
176,QUERY_DETAILS,2023-10-22 08:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
177,QUERY_DETAILS,2023-10-22 09:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
178,QUERY_DETAILS,2023-10-22 10:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0
179,QUERY_DETAILS,2023-10-22 11:00:00,mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm,database-2.cofhrj7zmyn4.eu-central-1.rds.amazo...,airbases,0.0


In [ ]:
import datetime
import time

 ############################################
query = """
SELECT 
  SUM("calls") AS sum_calls
FROM
  "test"."autogen"."QUERY_DETAILS"
WHERE
  time >= now() - 24h
  AND apiKey = 'mj1Vde9QC664608cJ24CV3Zf2Y9tdgzt20P8quOm'
  AND "host" = 'database-2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com'
  AND "db" = 'platform-v2'
 
GROUP BY
  time(1h),  "query_id"

"""
 # -- AND query_id = '998035682853375308'

start_time = datetime.datetime.now()
table = client.query(query=query, database=bucket, mode ="all", language="influxql")
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print(f"Time elapsed: {time_difference.total_seconds()} seconds")

dataframe = table.to_pandas() # This one automatically eliminitaes the NULL values. Not good. 
print

data_dict = {
    "Measurement": table[0],
    "Time": table[1],
    "apiKey":table[2],
    "host":table[3],
    # "db":table[4],
    # "query_id":table[5],
    # "Max": table[6]
}
# Create a Pandas DataFrame
df = pd.DataFrame(data_dict)
df

distinct_values_count = df['query_id'].nunique()
print(f"Number of distinct values in 'query_id': {distinct_values_count}")

# Close the client connection
# client.close()


Time elapsed: 8.002975 seconds


KeyError: 'query_id'

Read using InfluxQL

In [ ]:
%%bash
influx query \
'from(bucket: "Tables_Bucket")
  |> range(start: -14d, stop: -12d)
  |> filter(fn: (r) => r._measurement == "m" and r._field == "calls" and r.query_id == "qry1")
  |> aggregateWindow(every: 60m, fn: max)
  |> difference(nonNegative: true)
  |> drop(columns: ["_start", "_stop", "_measurement"])'

In [ ]:
# Define your InfluxDB connection details
#url = "http://localhost:8086"
#token = "your_influxdb_token"
#org = "your_organization"
bucket = "Tables_Bucket"

# SQL Query (NOT Flux)
query = """
SELECT query_id, max(calls)
FROM "queries"
WHERE time >= '2023-10-10T00:00:00Z'
GROUP by query_id, time(60m)
"""

## Query_2 Doesn't work
query_2 = """
SELECT derivative(max("calls"), 5m) AS "difference_calls"
FROM "queries"
WHERE time >= -14d AND time < -12d AND query_id = 'qry1'
GROUP BY time(60m) fill(null)
"""

token = os.environ.get("INFLUXDB_TOKEN")
token = "XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw=="
org = "Dev"
host = "https://us-east-1-1.aws.cloud2.influxdata.com"

client = InfluxDBClient3(host=host, token=token, org=org, ssl_ca_cert=certifi.where())
# You can bring only the schema to help troubleshooting
# schema = client.query(query=query, database="Tables_Bucket", mode="schema", language="influxql")
# print(schema)

table = client.query(query=query_2, database="Tables_Bucket", mode ="all", language="influxql")
dataframe = table.to_pandas() # This one automatically eliminitaes the NULL values. Not good. 
print (table)


# Close the client connection
# client.close()


pyarrow.Table
iox::measurement: string not null
time: timestamp[ns]
difference_calls: double
----
iox::measurement: []
time: []
difference_calls: []
